# 1) Importation des modules
On importe les différents modules que nous allons utiliser : tensorflow, numpy, pandas, etc..
Pour la création de notre modèle, nous utilisons des layers de keras
Nous utilisaons aussi des fonctions (sequence, pad_sequences, etc) de keras pour transformer nos phrases en vecteurs, matrices

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras

from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


On importe le jeu de données disponnible sur le drive (changer le chemin pour le faire fonctionner)

In [4]:
df = pd.read_csv('gdrive/MyDrive/Colab Notebooks/Projet/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

df.columns = ['target','id','date','flag','user','text']

Et on visualise le jeu de données

In [5]:
#Visualize dataframe
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


On peut voir qu'il y a 1600000 tweets, avec 6 colonnes (target(le sentiment), id du tweet, la date, flag, l'utilisateur qui a tweeté et le tweet (text))

In [6]:
print("Shape : " + str(df.shape))
print("Length : " + str(len(df)))

Shape : (1600000, 6)
Length : 1600000


Le seules colonnes qui nous intéressent sont celles du sentiment et le texte, on ne garde donc que celles-là dans notre jeu de données

In [7]:
#Let's keep only targets and text
df = df[['target','text']]
df

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


# 2) Traitement des tweets
Maintenant que nous avons ce qui nous intéresse, il va falloir traiter le texte

On commence par la fonction clean_tweet, qui va permettre d'enlever les liens (www., http:://), les mentions (@user), les zones vides, ainsi que les hashtags

On crée une nouvelle colonne dans notre dataframe, dans laquelle on mets le tweet "nettoyé"

In [8]:
import re
def clean_tweet(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text) #remove links
    text = re.sub('@[^\s]+',' ', text) #remove mentions
    text = re.sub('[\s]+', ' ', text) #remove blanks
    text = re.sub(r'#([^\s]+)', r' ', text) #remove #
    return text
df['clean_tweet'] = df.text.apply(clean_tweet)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,target,text,clean_tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- Awww, that's a bummer. You shoulda got Davi..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am..."
...,...,...,...
1599995,4,Just woke up. Having no school is the best fee...,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...,Happy 38th Birthday to my boo of alll time!!! ...


Ensuite, nous continuons le nettoyage des tweets en supprimant les ponctuations (on décide de ne garder que les lettres). Puis on mets toutes nos lettres en minuscule. Enfin, on supprime les stopwords. Les tweets étant en anglais, on utilise le dictionnaire de stopwords anglais.

On remplace les tweets de la colonne clean_tweets par les nouveaux tweets

In [9]:
#Remove punctiations, numbers, uppercases letters and stopwords in tweets
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
def remove_punct(text):
    text = text.str.replace('[^a-zA-Z#]',' ') # keep only letters, if not a letter replace with space
    text = text.str.lower() #remove uppercases letters
    text = text.apply(lambda x: ' '.join(w for w in x.split() if w not in stopWords)) # remove stopwords
    return text
df['clean_tweet'] = remove_punct(df['clean_tweet'])
df 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,target,text,clean_tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see
...,...,...,...
1599995,4,Just woke up. Having no school is the best fee...,woke school best feeling ever
1599996,4,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interviews
1599997,4,Are you ready for your MoJo Makeover? Ask me f...,ready mojo makeover ask details
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...,happy th birthday boo alll time tupac amaru sh...


L'étape suivante est le Stemming. Pour procéder, nous devons tokenizer chaque tweet en splitant simplement les mots dans les tweets

In [10]:
#tokenize texts
tokenized_text = df['clean_tweet'].apply(lambda x: x.split())
tokenized_text.head()

0    [awww, bummer, shoulda, got, david, carr, thir...
1    [upset, update, facebook, texting, might, cry,...
2    [dived, many, times, ball, managed, save, rest...
3              [whole, body, feels, itchy, like, fire]
4                                 [behaving, mad, see]
Name: clean_tweet, dtype: object

Puis nous appliquons le Stemming (PorterStemming) à nos tokens

In [11]:
#stemming words
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_text = tokenized_text.apply(lambda x: [stemmer.stem(i) for i in x])
tokenized_text.head()

0    [awww, bummer, shoulda, got, david, carr, thir...
1    [upset, updat, facebook, text, might, cri, res...
2    [dive, mani, time, ball, manag, save, rest, go...
3               [whole, bodi, feel, itchi, like, fire]
4                                    [behav, mad, see]
Name: clean_tweet, dtype: object

Nous créons une nouvelle colonne représentant les tokens + Stemming des tweets

In [12]:
df['tokens'] = tokenized_text
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,target,text,clean_tweet,tokens
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day,"[awww, bummer, shoulda, got, david, carr, thir..."
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...,"[upset, updat, facebook, text, might, cri, res..."
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds,"[dive, mani, time, ball, manag, save, rest, go..."
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,"[whole, bodi, feel, itchi, like, fire]"
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see,"[behav, mad, see]"
...,...,...,...,...
1599995,4,Just woke up. Having no school is the best fee...,woke school best feeling ever,"[woke, school, best, feel, ever]"
1599996,4,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interviews,"[thewdb, com, cool, hear, old, walt, interview]"
1599997,4,Are you ready for your MoJo Makeover? Ask me f...,ready mojo makeover ask details,"[readi, mojo, makeov, ask, detail]"
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...,happy th birthday boo alll time tupac amaru sh...,"[happi, th, birthday, boo, alll, time, tupac, ..."


Enfin, nous recréons les phrases en rassemblant tous les tokens. Nous obtenons donc les tweets finaux, qui vont nous servir pour l'entrainement.
Ces textes sont placés dans une nouvelle colonne, *clean_tweet_final*

In [13]:
# now let's see how clean tweets look like in the dataframe
for i in range(len(tokenized_text)):
    tokenized_text[i] = ' '.join(tokenized_text[i])

df['clean_tweet_final'] = tokenized_text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
df

,target,text,clean_tweet,tokens,clean_tweet_final
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day,"[awww, bummer, shoulda, got, david, carr, thir...",awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...,"[upset, updat, facebook, text, might, cri, res...",upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds,"[dive, mani, time, ball, manag, save, rest, go...",dive mani time ball manag save rest go bound
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,"[whole, bodi, feel, itchi, like, fire]",whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see,"[behav, mad, see]",behav mad see
...,...,...,...,...,...
1599995,4,Just woke up. Having no school is the best fee...,woke school best feeling ever,"[woke, school, best, feel, ever]",woke school best feel ever
1599996,4,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interviews,"[thewdb, com, cool, hear, old, walt, interview]",thewdb com cool hear old walt interview
1599997,4,Are you ready for your MoJo Makeover? Ask me f...,ready mojo makeover ask details,"[readi, mojo, makeov, ask, detail]",readi mojo makeov ask detail
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...,happy th birthday boo alll time tupac amaru sh...,"[happi, th, birthday, boo, alll, time, tupac, ...",happi th birthday boo alll time tupac amaru sh...


# 3) Préparation des données

Maintenant que nos données sont traitées, il faut les utiliser.

Pour commencer, on modifie 4 par 1 dans la colonne des sentiments, pour une meilleure lecture.
Ainsi 0 = sentiment négatif, 1 = sentiment positif.

Ensuite, nous sélectionnons les données positives et négatives en les mettant dans *data_pos* et *data_neg*

Pour nos tests, nous choisissons de ne sélectionner qu'une partie des données : 30000 tweets positifs, 30000 tweets négatifs.

*data* sera donc nos données d'entrainement

In [15]:
# Put label = 4 to 1 for better reading
df['target'][df['target']==4]=1

# and put pos and neg in data_pos and data_neg
#data_pos_full from 800000 to 1599999
data_pos = df[df['target'] == 1]
#data_neg_full from 0 to 799999
data_neg = df[df['target'] == 0]

# for training, keep only 20000 elements of neg and pos
data_neg_test = data_neg.iloc[:int(200000)] # 30000 neg tweets
data_pos_test = data_pos.iloc[:int(200000)] # 30000 pos tweets



# For tests we keep only 80000 elements
data = pd.concat([data_neg_test, data_pos_test]);

data=data[['clean_tweet_final', 'target']]
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


,clean_tweet_final,target
0,awww bummer shoulda got david carr third day,0
1,upset updat facebook text might cri result sch...,0
2,dive mani time ball manag save rest go bound,0
3,whole bodi feel itchi like fire,0
4,behav mad see,0
...,...,...
999995,thank need,1
999996,mayb,1
999997,hell window price rang unless free,1
999998,neah wish reminisc read post last tweet,1


A présent nous définissons nos données d'entrée : x sera les tweets et y les sentiments

In [16]:
#x = data.cleaned_tweet
x = data.clean_tweet_final # input features
y = data.target # label

*max_len* représente le nombre maximum de caractéristiques/mots sélectionnés pour l'entrainement. ces mots seront utilisés selon l'importance qui distingue les tweets positifs et négatifs. Nous en choisissons 500.

Ensuite, nous définissons le Tokenizer

In [17]:
# For training, we select maximum 500 features/words
max_len = 500
tokenizer = Tokenizer(num_words = 2000) # Tokenizer will use only 2000 most common words
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

vocab_size = len(tokenizer.word_index)
print("Total words", vocab_size)
print("Matrix shape", sequences_matrix.shape)


Total words 89623
Matrix shape (400000, 500)


In [18]:
x_train, x_test, y_train, y_test = train_test_split(sequences_matrix, y, test_size=0.3, random_state=2) # create train and test data
#x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.clean_tweet_final), maxlen=400)
#x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.clean_tweet_final), maxlen=400)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

print("x_train : " + str(x_train.shape))
print("y_train : " + str(y_train.shape))
print("x_test : " + str(x_test.shape))
print("y_test : " + str(y_test.shape))

x_train : (280000, 500)
y_train : (280000, 1)
x_test : (120000, 500)
y_test : (120000, 1)


# 4) Le modèle

Maintenant que nos données sont prêtes, nous allons devoir passer à la définition et à l'entrainement du modèle

In [19]:
def model1(): # Pour entrainer le modèle on utilise la fonction tensorflow_based_model
    model = Sequential()
    inputs = Input(name='inputs',shape=[max_len])# etape 1
    model.add(Embedding(vocab_size, 64, input_length=max_len)) # etape 2
    model.add(LSTM(64)) # etape3
    model.add(Dense(256,name='FC1')) # etape 4
    model.add(Activation('relu')) # etape 5
    model.add(Dropout(0.5)) # step6
    model.add(Dense(1,name='out_layer')) # etape 4 encore, mais on donne seulement 1 sortie car il faut classer le tweet positif ou négatif
    model.add(Activation('sigmoid')) # etape 5 encore mais la fonction d'activation est sigmoid pour une seule sortie
    #model = Model(inputs=inputs,outputs=layer) # Ici on obtient la sortie finale pour la classification
    return model # on retourne la valeur

In [20]:
from keras.models import Sequential
def model2():
  model = Sequential()
  inputs = Input(name='inputs',shape=[max_len])
  model.add(Embedding(2000,50,input_length=max_len))
  model.add(LSTM(64))
  model.add(Dense(512,name='FC1'))
  model.add(Activation('relu'))
  model.add(Dropout(0.75))
  model.add(Dense(256,name='FC2'))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(128,name='FC3'))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(50, activation='softmax'))
  model.add(Dense(1,name='out_layer'))
  model.add(Activation('sigmoid'))

  return model

In [21]:
def model1LSTM():
  model = Sequential()
  embedding_layer = Embedding(sequences_matrix.shape[0], 500, weights=[sequences_matrix], input_length=max_len , trainable=False)
  model.add(embedding_layer)
  model.add(LSTM(128))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  
  return model

In [22]:
model = model1() # on crée notre modèle
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) 

In [23]:
history=model.fit(x_train,y_train,batch_size=80,epochs=10, validation_split=0.1)# On entraine le modèle avec les données d'entraînement
print('Entrainement terminé !')

Epoch 1/10
3150/3150 [==============================] - 275s 85ms/step - loss: 0.5399 - accuracy: 0.7229 - val_loss: 0.4957 - val_accuracy: 0.7544
Epoch 2/10
1591/3150 [==============>...............] - ETA: 2:07 - loss: 0.4899 - accuracy: 0.7598

KeyboardInterrupt: ignored

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
plot_model_history(history)

In [ ]:
score = model.evaluate(x_test, y_test)
print("Accuracy :",score[1]) # précision du modèle

In [ ]:
def get_sentiment(score):
  if score < 0.5 :
    return "Negative sentiment"
  else : 
    return "Positive sentiment"

In [ ]:
def clean_phrase(text):
  text.replace('[^a-zA-Z#]',' ') # keep only letters
  text.lower() #remove uppercases letters
  #remove links, mentions, blanks and hashtags
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
  text = re.sub('@[^\s]+',' ', text)
  text = re.sub('[\s]+', ' ', text)
  text = re.sub(r'#([^\s]+)', r' ', text)
  text = ' '.join(word for word in text.split() if word not in stopWords) #remove stopwords
  tokens = text.split()
  tokensStem = [stemmer.stem(i) for i in tokens] # stemming
  textClean = ' '.join(tokensStem)
  
  return textClean

In [ ]:
clean_phrase("Hello I am in the bummer happy")

In [ ]:
def predict(text):
  # clean the text
  textClean = clean_phrase(text)
  # Tokenize text
  x_test = pad_sequences(tokenizer.texts_to_sequences([textClean]), maxlen=max_len)
  # Predict
  score = model.predict([x_test])[0]
  # Decode sentiment
  label = get_sentiment(score)

  return {"label": label, "score": float(score)}

In [ ]:
predict("I like the burger")

In [ ]:
predict("Covid is a really sad period")